In [1]:
import agate

open the Master.csv and Salaries.csv tables

In [2]:
master = agate.Table.from_csv('Master.csv') # File with player details
salary = agate.Table.from_csv('Salaries.csv') #File with baseball players' salaries

check to what type each item is with `print(table)`

In [3]:
print(master)
print(salary)

| column       | data_type |
| ------------ | --------- |
| playerID     | Text      |
| birthYear    | Number    |
| birthMonth   | Number    |
| birthDay     | Number    |
| birthCountry | Text      |
| birthState   | Text      |
| birthCity    | Text      |
| deathYear    | Number    |
| deathMonth   | Number    |
| deathDay     | Number    |
| deathCountry | Text      |
| deathState   | Text      |
| deathCity    | Text      |
| nameFirst    | Text      |
| nameLast     | Text      |
| nameGiven    | Text      |
| weight       | Number    |
| height       | Number    |
| bats         | Text      |
| throws       | Text      |
| debut        | Date      |
| finalGame    | Date      |
| retroID      | Text      |
| bbrefID      | Text      |

| column   | data_type |
| -------- | --------- |
| yearID   | Number    |
| teamID   | Text      |
| lgID     | Text      |
| playerID | Text      |
| salary   | Number    |



print out sample data with `table.print_table()`<br>
see additional options by pressing `ctrl + tab` over the `print_table()` method

In [4]:
master.print_table()
salary.print_table()

| playerID  | birthYear | birthMonth | birthDay | birthCountry | birthState | ... |
| --------- | --------- | ---------- | -------- | ------------ | ---------- | --- |
| aardsda01 |     1,981 |         12 |       27 | USA          | CO         | ... |
| aaronha01 |     1,934 |          2 |        5 | USA          | AL         | ... |
| aaronto01 |     1,939 |          8 |        5 | USA          | AL         | ... |
| aasedo01  |     1,954 |          9 |        8 | USA          | CA         | ... |
| abadan01  |     1,972 |          8 |       25 | USA          | FL         | ... |
| abadfe01  |     1,985 |         12 |       17 | D.R.         | La Romana  | ... |
| abadijo01 |     1,854 |         11 |        4 | USA          | PA         | ... |
| abbated01 |     1,877 |          4 |       15 | USA          | PA         | ... |
| abbeybe01 |     1,869 |         11 |       11 | USA          | VT         | ... |
| abbeych01 |     1,866 |         10 |       14 | USA          | NE         

join the two csv's

In [5]:
joined = master.join(salary)

see what columns the `joined` table contains

In [6]:
print(joined)

| column       | data_type |
| ------------ | --------- |
| playerID     | Text      |
| birthYear    | Number    |
| birthMonth   | Number    |
| birthDay     | Number    |
| birthCountry | Text      |
| birthState   | Text      |
| birthCity    | Text      |
| deathYear    | Number    |
| deathMonth   | Number    |
| deathDay     | Number    |
| deathCountry | Text      |
| deathState   | Text      |
| deathCity    | Text      |
| nameFirst    | Text      |
| nameLast     | Text      |
| nameGiven    | Text      |
| weight       | Number    |
| height       | Number    |
| bats         | Text      |
| throws       | Text      |
| debut        | Date      |
| finalGame    | Date      |
| retroID      | Text      |
| bbrefID      | Text      |
| yearID       | Number    |
| teamID       | Text      |
| lgID         | Text      |
| playerID2    | Text      |
| salary       | Number    |



check if all the players have a salary assigned. The easiest way is to deduct the length of the `joined` table from the `master` table

In [7]:
len(master) - len(joined)

0

!!! the join takes a (seemingly) random entry from the dataset,<br>
given the year range for `moyerja01` is 1986 - 2012

In [8]:
joined.where(lambda player: player["playerID"] == "moyerja01").select("yearID").print_table()

| yearID |
| ------ |
|  1,999 |


## What is the average (mean, median, max, min) salary?

In [9]:
joined.aggregate(agate.Mean('salary'))

Decimal('1466053.222805900456330255757')

In [10]:
joined.aggregate(agate.Median('salary'))

Decimal('438750')

In [11]:
joined.aggregate(agate.Max('salary'))

Decimal('26000000')

In [12]:
joined.aggregate(agate.Min('salary'))

Decimal('0')

## Who makes the most?

In [13]:
highest = joined.aggregate(agate.Max('salary'))

In [14]:
joined.where(lambda player: player["salary"] == highest).select(["nameFirst","nameLast","birthCountry","salary"]).print_table()

| nameFirst | nameLast | birthCountry |     salary |
| --------- | -------- | ------------ | ---------- |
| Frank     | Thompson |              | 26,000,000 |


## What are the most common baseball players salaries?

Draw a histogram. <br>
*([documentation reference](http://agate.readthedocs.io/en/1.5.5/cookbook/charting.html?highlight=pivot))*

In [15]:
binned_salaries = joined.bins('salary', 10)
binned_salaries.print_bars('salary')

salary                     Count
[0 - 3,000,000)           15,897 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                  
[3,000,000 - 6,000,000)    1,844 ▓░░░░░░░░                                                                              
[6,000,000 - 9,000,000)      662 ▓░░░                                                                                   
[9,000,000 - 12,000,000)     241 ▓░                                                                                     
[12,000,000 - 15,000,000)    129 ▓░                                                                                     
[15,000,000 - 18,000,000)     45 ▓                                                                                      
[18,000,000 - 21,000,000)     16 ▓                                                                                      
[21,000,000 - 24,000,000)     11 ▓                                                                                      

## Who are the top 10% highest-paid players?

calculate percentiles

In [16]:
percentiles = joined.aggregate(agate.Percentiles('salary'))

filter salaries larger or equal than the 90th percentile

In [17]:
top_ten_percent = joined.where(lambda r: r['salary'] >= percentiles[90])

order the data and `select` the information on the highest paid players

In [18]:
ordered = top_ten_percent.order_by('salary', reverse=True)
ordered.select(["nameFirst","nameLast","birthYear","birthState","salary"]).print_table()

| nameFirst  | nameLast   | birthYear | birthState   |     salary |
| ---------- | ---------- | --------- | ------------ | ---------- |
| Frank      | Thompson   |           |              | 26,000,000 |
| Gene       | Wright     |     1,878 | OH           | 23,428,571 |
| Ricky      | Wright     |     1,958 | TX           | 22,708,525 |
| Heathcliff | Slocumb    |     1,966 | NY           | 22,500,000 |
| Ribs       | Raney      |     1,923 | MI           | 22,000,000 |
| Nick       | Rumbelow   |     1,991 | TX           | 22,000,000 |
| Chris      | Short      |     1,937 | DE           | 22,000,000 |
| Bill       | Stafford   |     1,938 | NY           | 22,000,000 |
| Champ      | Summers    |     1,946 | WA           | 22,000,000 |
| Lou        | Tost       |     1,911 | WA           | 22,000,000 |
| Colin      | Ward       |     1,960 | CA           | 21,680,727 |
| Harry      | Wright     |     1,835 |              | 21,600,000 |
| Paul       | Waner      |     1,903 | OK      

make a bar chart of the 10 highest paid players

In [19]:
ordered.limit(10).print_bars("nameLast", 'salary')

nameLast     salary
Thompson 26,000,000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░             
Wright   23,428,571 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                      
Wright   22,708,525 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                        
Slocumb  22,500,000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                         
Raney    22,000,000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                          
Rumbelow 22,000,000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                          
Short    22,000,000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                          
Stafford 22,000,000 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                          
Summers  22,

save the data

In [20]:
top_ten_percent.to_csv('highest-paid.csv')